In [107]:
# import zip_longest from itertools
from itertools import zip_longest
import pandas as pd


In [143]:
# Print the number of lines in the file from the shell script
no_of_lines = ! wc -l "MetObjects.csv"
no_of_lines

['12108 MetObjects.csv']

In [98]:
def return_dict_from_csv_line(header, line):
    # zip them
    zipped_line = zip_longest(header, line, fillvalue=None)
    # Use dict comprehension to generate the final dict
    ret_dict = {kv[0]: kv[1] for kv in zipped_line}
    return ret_dict

In [216]:
lines=[]
with open("MetObjects.csv", "r", encoding="latin1") as fd:
    first_line = fd.readline()
    header = first_line.replace("\n","").split(",")
    for i, line in enumerate(fd):
        line = line.replace("\n", "").split(",")
        d = return_dict_from_csv_line(header, line)
        lines.append(d)

In [217]:
# Reading the first line into a dataframe

with open("MetObjects.csv", "r", encoding="latin1") as fd:
    first_line = fd.readline()
    header = first_line.replace("\n", "").split(",")
col_header = header

In [159]:
col_header

['"Object Number',
 'Is Highlight',
 'Is Public Domain',
 'Object ID',
 'Department',
 'Object Name',
 'Title',
 'Culture',
 'Period',
 'Dynasty',
 'Reign',
 'Portfolio',
 'Artist Role',
 'Artist Prefix',
 'Artist Display Name',
 'Artist Display Bio',
 'Artist Suffix',
 'Artist Alpha Sort',
 'Artist Nationality',
 'Artist Begin Date',
 'Artist End Date',
 'Object Date',
 'Object Begin Date',
 'Object End Date',
 'Medium',
 'Dimensions',
 'Credit Line',
 'Geography Type',
 'City',
 'State',
 'County',
 'Country',
 'Region',
 'Subregion',
 'Locale',
 'Locus',
 'Excavation',
 'River',
 'Classification',
 'Rights and Reproduction',
 'Link Resource',
 'Metadata Date',
 'Repository',
 'Tags"']

In [227]:
metobj_df = pd.DataFrame(data = lines, columns=col_header)
print(metobj_df.shape)

(12108, 44)


In [226]:
metobj_df.describe

<bound method NDFrame.describe of           "Object Number Is Highlight Is Public Domain Object ID  \
0            "1979.486.1        False            False         1   
1            "1980.264.5        False            False         2   
2              "67.265.9        False            False         3   
3             "67.265.10        False            False         4   
4             "67.265.11        False            False         5   
...                  ...          ...              ...       ...   
12103  "1974.356.1 recto        False            False     11814   
12104          "54.143.8        False            False     11815   
12105        "1976.201.4        False            False     11816   
12106            "64.118        False            False     11817   
12107                  4         None             None      None   

              Department Object Name                                Title  \
0      The American Wing        Coin         One-dollar Liberty Head Coi

In [229]:
# Filter out missing data
# Using how = all will drop rows that are all NA
# So after using the parameter it did not drop a single row
metobj_df_1 = metobj_df.dropna(how='all')
metobj_df_1.shape

(12108, 44)

In [230]:
# Removing duplicates
# After dropping the duplicates the new row count
metobj_df_1 = metobj_df.drop_duplicates()

In [245]:
bef4 = metobj_df.shape[0]
aftr = metobj_df_1.shape[0]
print("The row count before dropping duplicates is: {}.""\n""The row count after dropping the duplicates is: {}.".format(bef4,aftr))

The row count before dropping duplicates is: 12108.
The row count after dropping the duplicates is: 11923.


In [233]:
# So all rows that were duplicate are dropped from the actual dataframe 
# The new data frame with no duplicates is metobj_df_1
metobj_dedup = metobj_df_1

In [234]:
metobj_dedup.shape

(11923, 44)

In [246]:
metobj_dedup.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            12098, 12099, 12100, 12101, 12102, 12103, 12104, 12105, 12106,
            12107],
           dtype='int64', length=11923)